Компания купила крупный сервис для чтения книг по подписке. В базе данных компании информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта. Первая задача как аналитика — проанализировать базу данных. А именно:

1) Подготовить подключение к базе данных

2) Провести предобработку данных

3)Посчитать, сколько книг вышло после 1 января 2000 года

4)Для каждой книги посчитать количество обзоров и среднюю оценку

5)Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так можно исключить из анализа брошюры

6)Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

7)Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

## Подготовить подключение к базе данных

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
con = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
query = '''
            SELECT * FROM books
        '''
books = pd.io.sql.read_sql(query, con) 
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [3]:
query = '''
            SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(query, con) 
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [4]:
query = '''
            SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con) 
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [5]:
query = '''
            SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con) 
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [6]:
query = '''
            SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con) 
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


## Провести предобработку данных

In [7]:
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [8]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [9]:
books['publication_date'] = pd.to_datetime(books['publication_date'])

In [10]:
books.describe()

,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000


In [11]:
books.duplicated().sum()

0

In [12]:
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [13]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [14]:
authors.duplicated().sum()

0

In [15]:
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [16]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [17]:
publishers.duplicated().sum()

0

In [18]:
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [19]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [20]:
ratings.duplicated().sum()

0

In [21]:
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


In [22]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [23]:
reviews.duplicated().sum()

0

## Посчитать, сколько книг вышло после 1 января 2000 года


In [24]:
query = ''' select count(*)
from books
where publication_date > '2000-01-01'
'''

pd.read_sql(query, con)

,count
0,819


*После 1 января 2000 года вышла 819 книг*

## Для каждой книги посчитать количество обзоров и среднюю оценку

In [25]:
query = '''select 
books.book_id as book_id,
books.title as title,
avg(rating) as rating_avg,
count(distinct review_id) as review_cnt
from books
    left join ratings on ratings.book_id = books.book_id
    left join reviews on reviews.book_id = books.book_id
group by
    books.book_id,
    books.title
order by
    review_cnt desc
'''

pd.read_sql(query, con)

,book_id,title,rating_avg,review_cnt
0,948,Twilight (Twilight #1),3.662500,7
1,963,Water for Elephants,3.977273,6
2,734,The Glass Castle,4.206897,6
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
4,695,The Curious Incident of the Dog in the Night-Time,4.081081,6
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0
996,808,The Natural Way to Draw,3.000000,0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
998,221,Essential Tales and Poems,4.000000,0


*Количество обзоров разниться от 0 до 7. Самое большое количество имеет книга Стефани Майер "Сумерки". Средний рейтинг данной книги 3.6. Рейтинг в приложении имеет пятибальную систему*

## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так можно исключить из анализа брошюры

In [26]:
query = ''' select count(title) as title_cnt,
publisher
from publishers
    inner join books on books.publisher_id = publishers.publisher_id
where num_pages > 50
group by publisher
order by title_cnt desc
'''

pd.read_sql(query, con)

,title_cnt,publisher
0,42,Penguin Books
1,31,Vintage
2,25,Grand Central Publishing
3,24,Penguin Classics
4,19,Ballantine Books
...,...,...
329,1,Turtleback
330,1,Atheneum Books for Young Readers: Richard Jack...
331,1,Penguin Signet
332,1,Victor Gollancz


*Издательство Penguin Books выпустила наибольшее количество книг с суммой страниц больше 50*

## Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

In [27]:
query = ''' select 
authors.author as author,
avg(rating.rating_avg) as rating_avg,
count(rating.rating_cnt) as rating_cnt
from books
inner join(
            select
                book_id as book_id,
                count( distinct ratings.rating_id) as rating_cnt,
                avg(rating) as rating_avg
            from ratings
            group by book_id
            having
                count(ratings.rating_id) >= 50) as rating on rating.book_id = books.book_id
inner join authors on authors.author_id = books.author_id
group by author
order by rating_avg desc,
rating_cnt

'''

pd.read_sql(query, con)

,author,rating_avg,rating_cnt
0,J.K. Rowling/Mary GrandPré,4.283844,4
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,1
2,J.R.R. Tolkien,4.258446,2
3,Louisa May Alcott,4.192308,1
4,Rick Riordan,4.080645,1
5,William Golding,3.901408,1
6,J.D. Salinger,3.825581,1
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474,1
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879,1
9,Dan Brown,3.754540,2


*Учитывая только книги с 50 и более оценками, автором с самой высокой средней оценкой книг является Джоан Роулинг*

## Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [28]:
query ='''select
avg(reviwers.review_cnt) as review_cnt_avg
from
    (select
        count(text) as review_cnt
    from reviews
    where username in
        (select username
        from ratings
        group by
            username
        having
            count(ratings) > 50)
    group by 
            username
    ) as reviwers
'''

pd.read_sql(query, con)

,review_cnt_avg
0,24.333333


*Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет около 24 штук*

*По итогу работы выяснилось, что 819 книг было выпущенно с начала 21 века. Количество обзоров разниться от 0 до 7. Самое большое количество имеет книга Стефани Майер "Сумерки". Средний рейтинг данной книги 3.6. Рейтинг в приложении имеет пятибальную систему. Издательство Penguin Books выпустила наибольшее количество книг с суммой страниц больше 50, а автором с самой высокой средней оценкой книг является Джоан Роулинг. Пользователи в сервисе довольно активны - среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24.*